In [1]:
from tensorflow import keras
import keras.backend as K
from keras import regularizers
from keras import models
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical
import matplotlib.pyplot as pyplot
import os
import time
import tensorflow as tf
import pandas as pd
from numpy.random import seed
from csv import writer
from csv import reader


In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()  # import do data set mnist

In [3]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype('float32') / 255


train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [4]:
x_val = test_images[-10000:]
y_val = test_labels[-10000:]
x_train = train_images[:-10000]
y_train = train_labels[:-10000]

In [5]:
seeds = list(range(0,10))
multiply= [0.5,1,2]
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [ ]:
for mltp in multiply:
    for actual_seed in seeds:
        seed(actual_seed)
        tf.random.set_seed(actual_seed)
        random_state = actual_seed

        model = models.Sequential() 
        model.add(layers.Dense(int(512*mltp), activation= 'relu', input_shape=(28 * 28,)))
        model.add(layers.Dense(10, activation='softmax' ))    
        
        filename =  ".\Mnist_seed_adam_result/"+"Mnist_" + str(mltp) + "_" +str(actual_seed)+ ".csv"
        history_logger= keras.callbacks.CSVLogger(filename, separator=",", append= False)
        time_callback = TimeHistory()
        model.compile(optimizer="adam",
              loss='categorical_crossentropy')

        history = model.fit(x_train,
                            y_train,
                            epochs=200,
                            batch_size=128, 
                            validation_data=(x_val, y_val),
                            callbacks=[history_logger,time_callback])
        
        times = time_callback.times
        csv_input = pd.read_csv(filename)
        csv_input['time'] = times
        csv_input.to_csv(filename, index=False)
        K.clear_session()
        del model
        tf.compat.v1.reset_default_graph()
